# VacationPy


## Observable Trends and Insights

In this task, I understand that creating an exploratory-interactive geospatial, Weather information, heat maps, and visualization through Google API, and other similar API applications play a remarkable role to exploit the potential socio-economic value buried inside monotonously large datasets.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [22]:
# Load the csv exported in Part I to a DataFrame

weather_csv_file = "../output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,coahuayana,18.7333,-103.6833,87.04,45.0,30.0,4.94,MX,1.678901e+09
1,saskylakh,71.9167,114.0833,16.32,92.0,100.0,8.28,RU,1.678901e+09
2,ancud,-41.8697,-73.8203,57.06,52.0,12.0,4.50,CL,1.678901e+09
3,dubbo,-32.2500,148.6167,59.13,88.0,6.0,3.44,AU,1.678901e+09
4,marrakesh,31.6315,-8.0083,89.67,17.0,0.0,6.91,MA,1.678901e+09


# Humidity Heatmap

Configure gmaps.

Use the Lat and Lng as locations and Humidity as the weight.

Add Heatmap layer to map.

In [23]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.

In [11]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,hilo,19.7297,-155.0900,79.29,82.0,0.0,3.44,US,1.678901e+09
335,port hedland,-20.3167,118.5667,78.85,83.0,0.0,6.91,AU,1.678901e+09
336,cockburn town,21.4612,-71.1419,78.75,72.0,0.0,9.75,TC,1.678901e+09
448,richards bay,-28.7830,32.0377,73.38,85.0,0.0,5.64,ZA,1.678901e+09
502,salalah,17.0151,54.0924,78.89,69.0,0.0,3.44,OM,1.678901e+09
535,molina,-35.1167,-71.2833,78.53,29.0,0.0,4.61,CL,1.678901e+09


# Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [14]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 120: hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 335: port hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 336: cockburn town.
Closest hotel is Osprey Beach Hotel.
------------
Retrieving Results for Index 448: richards bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 502: salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
------------
Retrieving Results for Index 535: molina.
Closest hotel is Hostal Pucará.
------------


In [15]:
hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
120,hilo,19.7297,-155.0900,79.29,82.0,0.0,3.44,US,1.678901e+09,Hilo Hawaiian Hotel
335,port hedland,-20.3167,118.5667,78.85,83.0,0.0,6.91,AU,1.678901e+09,The Esplanade Hotel
336,cockburn town,21.4612,-71.1419,78.75,72.0,0.0,9.75,TC,1.678901e+09,Osprey Beach Hotel
448,richards bay,-28.7830,32.0377,73.38,85.0,0.0,5.64,ZA,1.678901e+09,BON Hotel Waterfront Richards Bay
502,salalah,17.0151,54.0924,78.89,69.0,0.0,3.44,OM,1.678901e+09,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
535,molina,-35.1167,-71.2833,78.53,29.0,0.0,4.61,CL,1.678901e+09,Hostal Pucará


In [16]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))